In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_core import SingleThreadedAgentRuntime
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

In [2]:
#load credentials
load_dotenv(override=True)

True

In [3]:
@dataclass
class Message:
    content:str

### Introduce Tool

In [4]:
serper=GoogleSerperAPIWrapper()
lanchain_tool=Tool(name="search_tool",func=serper.run,description="use tool when web search is needed")
autogen_tool=LangChainToolAdapter(lanchain_tool)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### Build Agents

In [ ]:
class Agent_1(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(
            name,
            model_client=model_client,
            tools=[autogen_tool],
            reflect_on_tool_use=True
        )
    
    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)


class Agent_2(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(
            name,
            model_client=model_client,
            tools=[autogen_tool],
            reflect_on_tool_use=True
        )
    
    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)


class Evaluator(RoutedAgent):
    def __init__(self, name: str):
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')
        self._delegate = AssistantAgent(name, model_client=model_client)
    
    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        # These must be defined elsewhere or passed in
        message_1 = Message(content=instruction1)
        message_2 = Message(content=instruction2)

        inner_1 = AgentId("Agent_1", "default")
        inner_2 = AgentId("Agent_2", "default")

        response1 = await self.send_message(message_1, inner_1)
        response2 = await self.send_message(message_2, inner_2)

        result = f"## pros of autogen:\n{response1.content} \n\n## cons of autogen:\n{response2.content}"
        decision_prompt = f"{judge}\n{result}\nRespond with your decision and a brief explanation."

        decision_message = TextMessage(content=decision_prompt, source="user")
        response = await self._delegate.on_messages([decision_message], ctx.cancellation_token)

        final_output = result + "\n\n## Decision:\n\n" + response.chat_message.content
        return Message(content=final_output)

In [7]:
runtime=SingleThreadedAgentRuntime()
await Agent_1.register(runtime,"Agent_1",lambda: Agent_1("Agent_1"))
await Agent_2.register(runtime,"Agent_2",lambda: Agent_2("Agent_2"))
await Evaluator.register(runtime,"evaluator",lambda: Evaluator("evaluator"))
runtime.start()

In [8]:
agent_id=AgentId("evaluator","default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
display(Markdown(response.content))

## pros of autogen:
Here are some reasons in favor of choosing AutoGen for your AI Agent project:

1. **Ease of Use**: AutoGen comes with built-in agents that can be directly utilized, streamlining the development process. The associated observability and debugging tools further simplify monitoring and controlling agent workflows.

2. **Scalability**: The modular and extensible nature of the framework allows for the creation of scalable and customizable systems, making it suitable for projects of various sizes and complexities.

3. **Multi-Agent System Support**: It excels at facilitating the creation of multi-agent systems, enabling AI assistants to collaborate and improve efficiency in completing tasks.

4. **Dynamic Environment Compatibility**: AutoGen is well-suited for complex automation in dynamic environments, enhancing its usefulness in varied applications.

5. **Human-in-the-Loop Workflows**: The framework supports human-in-the-loop methodologies, ensuring that human oversight can be integrated into AI processes, which can enhance reliability and user trust.

6. **Reduced Coordination Complexity**: By utilizing natural-language handoffs, AutoGen eliminates the need for custom inter-agent protocols, reducing coordination complexity among agents.

These advantages make AutoGen a compelling choice for developing AI agents, especially in projects that require flexibility, collaboration, and ease of integration. 

TERMINATE 

## cons of autogen:
Here are some cons of using AutoGen for an AI Agent project:

1. **Steep Learning Curve**: Users often report that AutoGen has a challenging learning curve. Its complex features and graph-based approach require a significant amount of time and effort to master.

2. **Limited Documentation**: The documentation for AutoGen has been criticized for being hard to read and lacking sufficient examples, which can hinder new users from effectively utilizing its capabilities.

3. **Technical Issues**: Some users have faced glitches and functionality problems with the interface. This can disrupt the user experience and lead to inefficiencies during development.

4. **Customization Needs**: Deploying AutoGen typically requires significant customization, which may increase the time and resources needed to implement the solution compared to other frameworks that offer easier out-of-the-box setups.

5. **Hosting and Infrastructure Management**: AutoGen may necessitate additional oversight regarding hosting and infrastructure management, which can add to the complexity and cost of deployment.

6. **Limited Creativity**: While AutoGen is good at generating content based on predefined inputs, it may be less flexible in terms of creativity and adaptability compared to other AI solutions.

These points should be considered when deciding whether AutoGen is right for your project. 

TERMINATE

## Decision:

Based on the research presented, I recommend using AutoGen for the project. The advantages of ease of use, scalability, support for multi-agent systems, and compatibility with dynamic environments significantly outweigh the drawbacks. While the steep learning curve and limited documentation could pose challenges, the potential for collaboration and reduced coordination complexity makes it a compelling choice for developing AI agents. The ability to integrate human oversight is also critical for ensuring reliability and user trust. Overall, AutoGen's strengths align well with the project's requirements.

TERMINATE

In [9]:
await runtime.stop()
await runtime.close()